# Labelbox Export and Convert

This notebook will: 
- downloaded the labels from a Labelbox project
- convert them into a format that works for training

## Labelbox Export

In [ ]:
pip install --upgrade fiftyone

In [ ]:
# Imports and configuration 
import fiftyone as fo

labelboxExportJson =  "/tf/notebooks/export-2021-02-01T01-34-34.538Z.json" # Download the exported JSON and update this
mediaDir = "/tf/export/planeBox"

In [ ]:
# Do the groundwork for importing, setup the dataset
import fiftyone.utils.labelbox as foul
from uuid import uuid4
labelbox_id_field = "labelbox_id"


# expect an error here if the dataset already exists
dataset = fo.Dataset(name="planeBox")
dataset.persistent = True

# This seems dumb, but it looks like you have to create a dummy record so you can have an instance with that field name
#sample = fo.Sample(filepath="./sample.jpg")
#dataset.add_samples([sample])
#for sample in dataset:
#    print(sample)
#    sample[labelbox_id_field] = str(uuid4())
#    sample.save()

In [ ]:
# Imports the Data from Labelbox into a Voxel51 Dataset
foul.import_from_labelbox(dataset, labelboxExportJson, download_dir=mediaDir, labelbox_id_field=labelbox_id_field)

In [ ]:
session = fo.launch_app(dataset, auto=False)

In [ ]:
# Add a label & tag that captures if the image was skipped, indicating there was no plane, or accepted, indicating there was a plane
from fiftyone import ViewField as F
label_field = "plane_ground_truth" 

model_view = dataset.exists("model")
for sample in model_view:
    sample[label_field] = fo.Classification(label="plane")
    sample.tags.append("plane")
    sample.save()


skipped_view = dataset.match({"model": {"$exists": False, "$eq": None}})
for sample in skipped_view:
    #print(sample)
    sample[label_field] = fo.Classification(label="noplane")
    sample.tags.append("noPlane")
    sample.save()    

In [ ]:
# This exports the images into a format that Keras is happy with
# It will create a dir structure based on the plane_ground_truth label
# images with a Plane in them will be placed in one DIR and those without will be placed in another
export_dir = "./export"
 # for example

# Export the dataset
dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    label_field=label_field,
)